# 🔴 Lab 3 — Inference Attack (Membership Inference)
### Certified AI Penetration Tester – Red Team (CAIPT-RT)

---

## 🎯 The Story

A hospital trained a machine learning model on real patient records — sensitive data including age, family background, financial situation, and health history. The hospital never releases the records themselves, but they do offer the trained model as a public tool: you send it a patient profile, it sends back a risk prediction.

You are an attacker. You do not have the patient records. But you have access to the model. Can you figure out **whether a specific person's data was used to train it?**

If you can, you have violated that person's privacy — you now know they were a patient at this hospital and their data was part of a sensitive medical study. That is a real legal and ethical violation even if you never saw the actual record.

This is called a **Membership Inference Attack**.

---

## 📖 What is a Membership Inference Attack?

It tries to determine whether a specific data point was part of a model's training dataset.

Think of it this way: imagine a chef who memorised certain recipes by cooking them over and over. If you asked that chef to recreate a dish, they would do it much more accurately for dishes they personally practised than for dishes they only vaguely know. A smart observer could figure out which dishes the chef trained on just by watching how confident they are.

Machine learning models behave the same way. They tend to be **more confident and make fewer mistakes on data they trained on** compared to data they have never seen. An attacker exploits this subtle difference.

**Real world examples:**
- Determining if a specific person's medical record was in a clinical trial dataset
- Confirming if someone's financial data trained a credit scoring model
- Violating GDPR or HIPAA by inferring membership in sensitive datasets

---

## 🗂️ What We Will Do in This Lab

1. Load the Nursery dataset — a realistic dataset of private family records
2. Prepare and encode the data so the model can understand it
3. Train a classifier and measure the gap between training and test performance
4. Run a simple rule-based membership inference attack
5. Run a more sophisticated black-box attack that trains its own attack model
6. Understand why the gap between training and test accuracy is the root cause

---

## ⚙️ Step 1: Import the Tools We Need

In [ ]:
# =============================================================================
# IMPORTS
# =============================================================================
# numpy      : handles numbers and arrays — the fundamental math library
# pandas     : loads and manages datasets like a spreadsheet in Python
# matplotlib : draws charts and graphs
# sklearn    : builds and trains machine learning models
# art        : Adversarial Robustness Toolbox — the attack toolkit
#
# We suppress the PyTorch warning from ART. PyTorch is an optional
# deep learning framework that we are not using in this lab.
# =============================================================================

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from art.estimators.classification import SklearnClassifier
from art.attacks.inference.membership_inference import (
    MembershipInferenceBlackBox,
    MembershipInferenceBlackBoxRuleBased
)
from art.utils import to_categorical

np.random.seed(42)
print("All tools imported successfully.")

---

## 📂 Step 2: Load and Understand the Nursery Dataset

We are using the **Nursery dataset** from the UCI Machine Learning Repository. It was originally created to rank applications for nursery school enrollment in the 1980s in Ljubljana, Slovenia.

Each row in this dataset represents a **family application**. The columns capture private information about each family — their financial situation, housing conditions, number of children, health status, and social circumstances.

For our attack scenario, imagine each row is a patient record at a hospital, and the model was trained on these records to make enrollment or eligibility decisions. The records are private. The model is public.

**Why this dataset?** Because it has real categorical structure (words, not just numbers), multiple classes to predict, and enough rows (nearly 13,000) to make a realistic attack experiment.

In [ ]:
# =============================================================================
# LOAD THE NURSERY DATASET
# =============================================================================
# The file has no header row — we supply the column names ourselves
# based on the UCI dataset documentation.
#
# Each row represents one family application with 8 features and a target.
# =============================================================================

column_names = [
    'parents',   # Parents' occupation: usual, pretentious, great_pret
    'has_nurs',  # Quality of the nursery: proper, less_proper, improper, critical, very_crit
    'form',      # Family form: complete, completed, incomplete, foster
    'children',  # Number of children: 1, 2, 3, more
    'housing',   # Housing conditions: convenient, less_conv, critical
    'finance',   # Financial standing: convenient, inconv
    'social',    # Social conditions: nonprob, slightly_prob, problematic
    'health',    # Health conditions: recommended, priority, not_recom
    'target'     # Enrollment decision: recommend, priority, not_recom, very_recom, spec_prior
]

df = pd.read_csv(
    '../datasets/nursery.data',
    header=None,
    names=column_names
)

print(f"Dataset loaded successfully.")
print(f"Total records : {len(df)}")
print(f"Total columns : {len(df.columns)}")
print("")
print("First 5 records:")
print("-" * 70)
print(df.head().to_string())
print("")
print("How many applications received each decision:")
print(df['target'].value_counts().to_string())

### 👀 What Do You See?

- Each row is a family. Each column is a piece of private information about them.
- The `target` column is what the model learned to predict — the enrollment decision.
- Notice that most applications were marked `not_recom` (not recommended). This imbalance matters — a model that just always guesses `not_recom` would be very accurate but completely useless.

**Think about this:** In a real hospital scenario, each of these rows might be a patient. The `target` might be a diagnosis. If an attacker could determine that a specific person's row was in this dataset, they would know that person received medical care at this institution — which is private information.

---

## 🔢 Step 3: Convert Text to Numbers (Encoding)

Machine learning models cannot read words. They only understand numbers. Every piece of text in this dataset must be converted into a number before training can begin.

We use a technique called **Label Encoding**. It assigns a unique integer to each unique text value in a column. For example:

- `finance` column: `convenient` → 0, `inconv` → 1
- `health` column: `not_recom` → 0, `priority` → 1, `recommended` → 2

This is purely a technical translation — it does not change the meaning of the data, it just puts it in a form the model can do mathematics with.

**Important:** After encoding, `X` is the input data (the 8 feature columns) and `y` is the target (the enrollment decision we want to predict).

In [ ]:
# =============================================================================
# LABEL ENCODING — CONVERT TEXT TO NUMBERS
# =============================================================================
# LabelEncoder.fit_transform() does two things in one step:
#   fit()      : scans the column and finds all unique values
#   transform(): replaces each unique value with its assigned integer
#
# We apply this to every column including the target column.
# =============================================================================

df_encoded = df.copy()
encoders = {}

for column in df_encoded.columns:
    le = LabelEncoder()
    df_encoded[column] = le.fit_transform(df_encoded[column])
    encoders[column] = le

X = df_encoded.drop('target', axis=1).values  # 8 input features as a 2D array
y = df_encoded['target'].values                # target labels as a 1D array

print("Encoding complete.")
print("")
print(f"X shape : {X.shape}")
print(f"  -> {X.shape[0]} records, each described by {X.shape[1]} numbers")
print("")
print(f"y shape : {y.shape}")
print(f"  -> {y.shape[0]} labels, one per record")
print("")
print(f"Unique target classes in y: {np.unique(y)}")
print(f"That represents {len(np.unique(y))} possible decisions the model can make")
print("")
print("Example — first record before and after encoding:")
print(f"  Original : {df.iloc[0].to_dict()}")
print(f"  Encoded  : {df_encoded.iloc[0].to_dict()}")

### 👀 What Do You See?

- Each record is now a row of 8 numbers instead of 8 words.
- `y` contains values 0 through 4, each representing one of the 5 enrollment decisions.
- The encoding is purely mechanical — the model does not know that `not_recom` means anything, it just learns that records with certain number patterns tend to get label 0.

---

## ✂️ Step 4: Split Data into Training and Testing Sets

Before training the model, we split our data into two groups:

**Training set** — the records the model learns from. In our attack scenario, we call these the **members**. These are the private records that went into the model.

**Test set** — records the model has never seen during training. We call these the **non-members**. The model has no memory of these at all.

The whole point of a membership inference attack is to figure out which group a given record belongs to — **without ever seeing the split directly**. We only have the model and the ability to query it.

**Why `stratify=y`?** This ensures that the same proportion of each target class appears in both the training and test sets. Without this, by pure chance you might end up with all the rare `very_recom` records landing in training and none in the test set, which would make the split unrepresentative.

In [ ]:
# =============================================================================
# TRAIN / TEST SPLIT
# =============================================================================
# test_size=0.3  : 70% of records go to training, 30% to testing
# random_state=42: sets the random seed so everyone gets the same split
# stratify=y     : ensures class proportions are the same in both sets
#
# TERMINOLOGY FOR THIS LAB:
#   X_train / y_train = MEMBERS   (the model will learn from these)
#   X_test  / y_test  = NON-MEMBERS (the model never sees these)
# =============================================================================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print("Data split complete.")
print("")
print(f"MEMBERS (training set)     : {len(X_train):,} records")
print(f"NON-MEMBERS (test set)     : {len(X_test):,} records")
print("")

# Verify the class balance is preserved in both sets
print("Class distribution check (should be similar in both sets):")
print(f"  Training - unique classes: {np.unique(y_train)}  counts: {[sum(y_train==c) for c in np.unique(y_train)]}")
print(f"  Testing  - unique classes: {np.unique(y_test)}   counts: {[sum(y_test==c) for c in np.unique(y_test)]}")
print("")
print("The attack will try to figure out which set each record belongs to,")
print("using only the model's behaviour — not this split directly.")

---

## 🏋️ Step 5: Train the Target Model

We train a **Random Forest** classifier. A Random Forest is an ensemble model — it builds many individual decision trees and combines their votes into a final prediction.

**What is a decision tree?** Imagine a flowchart of yes/no questions: "Is the family's finance status convenient? If yes, go left. If no, go right." Each question narrows down which enrollment category the family likely belongs to.

**Why a forest instead of one tree?** One tree can memorise the training data very specifically and make poor predictions on new data. Having hundreds of trees each trained on slightly different samples, then voting on the answer, produces a much more reliable result. This is the same principle as asking 100 experts instead of 1.

**The parameter `n_estimators=100`** means we build 100 trees. More trees generally means better accuracy but slower training.

**The critical thing to watch:** After training, we measure accuracy on both the training set and the test set. The difference between these two numbers is what makes membership inference possible.

In [ ]:
# =============================================================================
# TRAIN THE TARGET MODEL (RANDOM FOREST)
# =============================================================================
# RandomForestClassifier builds n_estimators independent decision trees.
# Each tree votes on the class label, and the majority vote wins.
#
# fit(X_train, y_train) is where training happens:
#   - X_train : the 8-feature input vectors for each training record
#   - y_train : the correct label (0-4) for each record
#
# The model adjusts its internal structure until it can map inputs to labels
# correctly. This is the learning step.
# =============================================================================

print("Training target model (100 decision trees)...")
print("This may take 15-20 seconds.")
print("")

target_model = RandomForestClassifier(n_estimators=100, random_state=42)
target_model.fit(X_train, y_train)

# Measure accuracy on training data (members)
train_accuracy = accuracy_score(y_train, target_model.predict(X_train))

# Measure accuracy on test data (non-members — never seen during training)
test_accuracy = accuracy_score(y_test, target_model.predict(X_test))

gap = train_accuracy - test_accuracy

print("Training complete!")
print("")
print("=" * 55)
print(f"Accuracy on MEMBERS (training data)   : {train_accuracy*100:.2f}%")
print(f"Accuracy on NON-MEMBERS (test data)   : {test_accuracy*100:.2f}%")
print(f"Gap                                   : {gap*100:.2f}%")
print("=" * 55)
print("")
print("What this gap means:")
print("  The model performs better on records it has seen before.")
print("  This is because it has partially memorised them during training.")
print("  An attacker can exploit this difference — if the model is more")
print("  confident and accurate on a record, it was probably a member.")

### 👀 What Do You See?

The model performs better on training data (members) than on test data (non-members). This is called **overfitting** — the model has partially memorised its training examples rather than just learning general patterns.

**Why does overfitting enable attacks?**

Think of it like a student who memorised last year's exam answers word for word. Ask them a question from last year's exam and they answer instantly with high confidence. Ask them a new question on the same topic and they are less sure. A smart observer watching this student could figure out which questions came from the old exam just by watching their confidence level.

The model behaves the same way:
- Member record → model predicts with high confidence → likely correct
- Non-member record → model is slightly less sure → slightly more likely to be wrong

The attack exploits this signal.

**The larger the gap, the easier the attack.** A perfectly generalising model with zero gap would give the attacker nothing to work with.

---

## 🔴 Step 6: Rule-Based Membership Inference Attack

The first attack is beautifully simple. It uses just one rule:

> **If the model predicts the correct label for a record → guess it is a member.**
> **If the model predicts incorrectly → guess it is a non-member.**

This works because models are more accurate on data they trained on. It is not a perfect attack — the model does sometimes get non-member records right too — but it performs better than random guessing, which is the baseline we care about.

**Why is 50% the baseline?** Because random guessing on a binary question (member vs non-member) gives 50% accuracy by definition. Any attack that scores above 50% is learning something real about the model's behaviour. Even 55% accuracy is a privacy violation in a sensitive context — it means the attacker is gaining real information.

In [ ]:
# =============================================================================
# RULE-BASED MEMBERSHIP INFERENCE ATTACK
# =============================================================================
# This attack implements one simple rule:
#   correct prediction → infer "member"
#   wrong prediction   → infer "non-member"
#
# We wrap our model in ART's SklearnClassifier so ART can work with it.
#
# IMPORTANT — why to_categorical()?
# ART's infer() function internally converts labels to one-hot format.
# One-hot means: for 5 classes, label 3 becomes [0, 0, 0, 1, 0].
# ART infers the number of classes from the sample passed in — but if
# our sample of 200 happens to be missing one class by chance, ART
# builds an array that is too small, and crashes with an IndexError.
#
# The fix: we convert to one-hot ourselves first, using the FULL class
# count from the complete dataset. This way ART always sees the right size.
# =============================================================================

art_model   = SklearnClassifier(model=target_model)
rule_attack = MembershipInferenceBlackBoxRuleBased(art_model)

# nb_classes = total number of unique target classes in the full dataset
# We compute this from y (full dataset) NOT from the sample
nb_classes  = len(np.unique(y))
sample_size = 200

print(f"Total classes in dataset : {nb_classes}")
print(f"Sample size for attack   : {sample_size} members + {sample_size} non-members")
print("")

# Draw a random sample of members (from training set)
member_idx = np.random.choice(len(X_train), sample_size, replace=False)
X_member   = X_train[member_idx]
y_member   = y_train[member_idx]

# Draw a random sample of non-members (from test set)
nonmember_idx = np.random.choice(len(X_test), sample_size, replace=False)
X_nonmember   = X_test[nonmember_idx]
y_nonmember   = y_test[nonmember_idx]

print(f"Classes present in member sample    : {np.unique(y_member)}")
print(f"Classes present in non-member sample: {np.unique(y_nonmember)}")
print("")

# Convert labels to one-hot using FULL class count (the key fix)
# to_categorical(labels, nb_classes) turns integer labels into binary rows
# Example with 5 classes: label 2 → [0, 0, 1, 0, 0]
y_member_oh    = to_categorical(y_member,    nb_classes)
y_nonmember_oh = to_categorical(y_nonmember, nb_classes)

print(f"One-hot shape for members    : {y_member_oh.shape}  (200 records × {nb_classes} classes)")
print(f"One-hot shape for non-members: {y_nonmember_oh.shape}")
print("")

# Run the attack
# infer() returns 1 for each record it thinks is a member, 0 for non-member
member_inferred    = rule_attack.infer(X_member,    y_member_oh)
nonmember_inferred = rule_attack.infer(X_nonmember, y_nonmember_oh)

# How often did it correctly identify members as members?
member_accuracy    = np.mean(member_inferred == 1)
# How often did it correctly identify non-members as non-members?
nonmember_accuracy = np.mean(nonmember_inferred == 0)
# Overall = average of both
overall_accuracy   = (member_accuracy + nonmember_accuracy) / 2

print("Rule-Based Attack Results:")
print("=" * 55)
print(f"Correctly identified members     : {member_accuracy*100:.1f}%")
print(f"Correctly identified non-members : {nonmember_accuracy*100:.1f}%")
print(f"Overall attack accuracy          : {overall_accuracy*100:.1f}%")
print("")
print(f"Random guessing baseline         : 50.0%")
print(f"Advantage over random guessing   : +{(overall_accuracy - 0.5)*100:.1f}%")

### 👀 What Do You See?

- **Random guessing gives 50%.** Any score above this means the attack is learning real information.
- Even a small advantage — say 55% or 60% — is meaningful in privacy terms. If someone's medical record was in a hospital's AI training set, you now have better-than-chance odds of knowing that.
- The rule-based attack is simple and fast but limited. It only uses whether the prediction was right or wrong — it ignores *how* confident the model was.

### 🧪 Try This

After running the rule-based attack, think about this: what if the model was equally accurate on members and non-members? The rule would not work at all. This is why privacy researchers recommend training models that generalise well — the attack surface shrinks as the accuracy gap shrinks.

---

## 🔴 Step 7: Black-Box Membership Inference Attack

The second attack is more sophisticated. Instead of a simple rule, it **trains its own small model** — called the attack model — to distinguish members from non-members.

**How does it work?**

The key insight is that when a model predicts on a record, it does not just output one label — it outputs a **probability distribution** across all possible classes. For example: `[0.02, 0.01, 0.91, 0.03, 0.03]`. This says the model is 91% confident the record belongs to class 2.

Members tend to get **higher confidence scores** on their true class. Non-members tend to get more spread-out, uncertain distributions. The attack model learns to tell these two patterns apart.

**This is called a black-box attack** because the attacker never sees inside the victim model — no code, no weights, no training data. They only see inputs and outputs, exactly as if they were calling a real API.

**The attack model is trained in two phases:**
1. Query the victim model on known members and known non-members to collect (confidence, true label) pairs
2. Train a small classifier to predict member/non-member from those confidence patterns

The attacker then uses this trained attack model to classify unknown records.

In [ ]:
# =============================================================================
# BLACK-BOX MEMBERSHIP INFERENCE ATTACK
# =============================================================================
# MembershipInferenceBlackBox works in two phases:
#
# Phase 1 — fit():
#   - Takes some known members and known non-members
#   - Queries the victim model on all of them
#   - Collects the victim's output probabilities for each record
#   - Trains a small Random Forest (the attack model) to learn:
#     high confidence on true class → likely member
#     lower, more uncertain confidence → likely non-member
#
# Phase 2 — infer():
#   - Takes new records of unknown membership
#   - Queries the victim model for their probabilities
#   - Feeds those probabilities to the trained attack model
#   - Returns 1 (member) or 0 (non-member) for each record
#
# The key fix applied here is the same as above: we convert labels to
# one-hot using the FULL class count before passing them to ART, to
# prevent the IndexError caused by ART miscounting classes from a sample.
# =============================================================================

print("Running black-box membership inference attack...")
print("Phase 1: Training the attack model on known members and non-members...")
print("(May take 15-30 seconds)")
print("")

bb_attack = MembershipInferenceBlackBox(art_model, attack_model_type='rf')

# We use the first half of our samples to TRAIN the attack model
# and the second half to TEST it (evaluate how well it generalised)
train_split = sample_size // 2  # 100 for training, 100 for testing

# Convert all four label arrays to one-hot using the FULL class count
# Each of these will be (100, 5) arrays — 100 records × 5 classes
y_member_train_oh    = to_categorical(y_member[:train_split],    nb_classes)
y_nonmember_train_oh = to_categorical(y_nonmember[:train_split], nb_classes)
y_member_test_oh     = to_categorical(y_member[train_split:],    nb_classes)
y_nonmember_test_oh  = to_categorical(y_nonmember[train_split:], nb_classes)

print(f"Attack model training: {train_split} members + {train_split} non-members")
print(f"Attack model testing : {train_split} members + {train_split} non-members (held out)")
print(f"Label array shapes   : {y_member_train_oh.shape} each")
print("")

# fit() trains the attack model on the known labelled examples
bb_attack.fit(
    x      = X_member[:train_split],        # known member inputs
    y      = y_member_train_oh,             # their one-hot labels
    x_test = X_nonmember[:train_split],     # known non-member inputs
    y_test = y_nonmember_train_oh           # their one-hot labels
)

print("Phase 2: Using the trained attack model to classify held-out records...")
print("")

# infer() uses the attack model on the held-out second half
bb_member_inferred    = bb_attack.infer(X_member[train_split:],    y_member_test_oh)
bb_nonmember_inferred = bb_attack.infer(X_nonmember[train_split:], y_nonmember_test_oh)

bb_member_acc    = np.mean(bb_member_inferred == 1)
bb_nonmember_acc = np.mean(bb_nonmember_inferred == 0)
bb_overall       = (bb_member_acc + bb_nonmember_acc) / 2

print("Black-Box Attack Results:")
print("=" * 55)
print(f"Correctly identified members     : {bb_member_acc*100:.1f}%")
print(f"Correctly identified non-members : {bb_nonmember_acc*100:.1f}%")
print(f"Overall attack accuracy          : {bb_overall*100:.1f}%")
print("")
print(f"Random guessing baseline         : 50.0%")
print(f"Advantage over random guessing   : +{(bb_overall - 0.5)*100:.1f}%")
print("")
print("Comparison:")
print(f"  Rule-based attack    : {overall_accuracy*100:.1f}%")
print(f"  Black-box attack     : {bb_overall*100:.1f}%")
print(f"  Improvement          : +{(bb_overall - overall_accuracy)*100:.1f}%")

### 👀 What Do You See?

- The black-box attack uses richer information — the full probability distribution — rather than just whether the prediction was right or wrong. This usually produces a better result.
- Compare the two attacks. How much did the extra sophistication help?

**On confidence distributions — why do they leak membership?**

When a model trains on a record it tends to push its confidence for the correct class very high — sometimes to 0.95 or 0.99. On a non-member record it has never seen, the confidence is more spread out. Even a small systematic difference across many records is enough for the attack model to learn the pattern.

### 🧪 Try This

Go back to the model training step and change `n_estimators=100` to `n_estimators=500`. Retrain and rerun the attacks.

- Does a more powerful model with less overfitting become harder to attack?
- What does this tell you about the relationship between model quality and privacy risk?

---

## 💭 Step 8: Reflect on What You Have Learned

In [ ]:
# =============================================================================
# REFLECTION — edit your answers below and run to save them
# =============================================================================

reflection = """
LAB 3 - INFERENCE ATTACK REFLECTION
=====================================

Q1: In plain English, what is a membership inference attack?
    What does the attacker learn, and why is it a privacy violation
    even if they never see the actual record?
A1: [TYPE YOUR ANSWER HERE]

Q2: The attack worked because the model had a training/test accuracy gap.
    What is overfitting in plain English, and why does it create this gap?
    Why does a larger gap make the attack easier?
A2: [TYPE YOUR ANSWER HERE]

Q3: The rule-based attack only used whether the prediction was correct.
    The black-box attack used the full probability distribution.
    Why does using probabilities give the attacker more information?
A3: [TYPE YOUR ANSWER HERE]

Q4: Even a small advantage above 50% is a serious privacy problem in
    medical or financial contexts. Give one concrete real-world example
    of harm that could result from a successful membership inference attack.
A4: [TYPE YOUR ANSWER HERE]

Q5: What defensive measures could reduce membership inference risk?
    Think about both technical controls (differential privacy, regularisation)
    and operational controls (who can query the model, how many times).
A5: [TYPE YOUR ANSWER HERE]

Q6: Which privacy regulation (GDPR, HIPAA, CCPA, etc.) would apply if
    an attacker revealed that someone's medical data was in a hospital's
    AI training set? What legal consequences could the hospital face?
A6: [TYPE YOUR ANSWER HERE]
"""

with open('../outputs/Lab3_Reflection.txt', 'w') as f:
    f.write(reflection)

print("Reflection saved to outputs/Lab3_Reflection.txt")
print(reflection)

---

## ✅ Lab 3 Complete

You have trained a real classifier, observed the overfitting gap, and performed two membership inference attacks — one simple and one sophisticated — using the Adversarial Robustness Toolbox.

The key lesson: **any model that behaves differently on its training data than on new data is potentially leaking privacy information**. The attacker does not need access to the training data or model internals. The model's own output probabilities are enough.

Return to [START_HERE.ipynb](START_HERE.ipynb) and open Lab 4 — Extraction Attack.

---
*Built with the Adversarial Robustness Toolbox (ART) — https://github.com/Trusted-AI/adversarial-robustness-toolbox*